# 3. STRUCTURED

In [1]:
import pandas as pd
import pandasql as psql
import numpy as np

In [2]:
path_cleansed = 'C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\UniversityHack2024\\data\\02_cleansed\\'
path_structured = 'C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\UniversityHack2024\\data\\03_structured\\'

## 3.1 PREINÓCULO

In [ ]:
path_preinoculo_02 = path_cleansed + 'Fases producción_Preinóculo.xlsx'

df_preinoculo_02 = pd.read_excel(path_preinoculo_02)

In [ ]:
# creamos columna de tiempo en horas
df_preinoculo_02['horas_totales_preinoculo'] = (df_preinoculo_02['Fecha/hora fin'] - df_preinoculo_02['Fecha/hora inicio']).dt.total_seconds() / 3600
# eliminamos las fechas
df_preinoculo_02 = df_preinoculo_02.drop(['Fecha/hora inicio', 'Fecha/hora fin'], axis=1)

In [ ]:
# Crear listas para almacenar los pH y turbidez seleccionados
ph_usados = []
turbidez_usados = []

# Iterar sobre cada fila del DataFrame
for i, row in df_preinoculo_02.iterrows():
    # Crear un DataFrame temporal con los pH, turbidez y líneas utilizadas de esa fila
    temp_df = pd.DataFrame({
        'ph': [row['ph_línea 1'], row['ph_línea 2'], row['ph_línea 3']],
        'turbidez': [row['turbidez_línea 1'], row['turbidez_línea 2'], row['turbidez_línea 3']],
        'utilizada': [row['linea_utilizada_línea 1'], row['linea_utilizada_línea 2'], row['linea_utilizada_línea 3']]
    })
    
    # Filtrar los tubos que se utilizan
    tubos_usados = temp_df[temp_df['utilizada'] == 1]
    
    # Tomar los valores de pH y turbidez de los tubos utilizados
    ph_usados.append(tubos_usados['ph'].values)
    turbidez_usados.append(tubos_usados['turbidez'].values)

# Agregar los valores seleccionados al DataFrame original
df_preinoculo_02['ph_usado_1'] = [x[0] if len(x) > 0 else np.nan for x in ph_usados]
df_preinoculo_02['ph_usado_2'] = [x[1] if len(x) > 1 else np.nan for x in ph_usados]
df_preinoculo_02['turbidez_usada_1'] = [x[0] if len(x) > 0 else np.nan for x in turbidez_usados]
df_preinoculo_02['turbidez_usada_2'] = [x[1] if len(x) > 1 else np.nan for x in turbidez_usados]

In [ ]:
df_preinoculo_03 = df_preinoculo_02.drop(['ph_línea 1', 'ph_línea 2', 'ph_línea 3', 'turbidez_línea 1', 'turbidez_línea 2', 'turbidez_línea 3', 'linea_utilizada_línea 1', 'linea_utilizada_línea 2', 'linea_utilizada_línea 3'], axis=1)

In [ ]:
df_preinoculo_03.to_excel(f'{path_structured}preinoculo_total.xlsx', index=False)

## 3.2 INÓCULO

In [ ]:
path_inoculoIPC_02 = path_cleansed + 'CineticosIPC_Inoculos.xlsx'
path_inoculo_02 = path_cleansed + 'Fases producción_Inóculo.xlsx'
path_biopequenyos_02 = path_cleansed + 'biorreactores_pequeños.xlsx'

df_inoculoIPC_02 = pd.read_excel(path_inoculoIPC_02)
df_inoculo_02 = pd.read_excel(path_inoculo_02)
df_biopequenyos_02 = pd.read_excel(path_biopequenyos_02)

In [ ]:
df_agrupado = df_inoculoIPC_02.groupby('Lote').agg({
    'Turbidez': ['mean', 'std', 'min', 'max'],
    'Viabilidad': ['mean', 'std', 'min', 'max']
}).reset_index()

In [ ]:
# renombramos columnas
df_agrupado.columns = ['_'.join(col).strip() for col in df_agrupado.columns.values]
df_agrupado = df_agrupado.rename(columns={'Lote_': 'LOTE'})

In [ ]:
df_final = pd.merge(df_inoculo_02, df_agrupado, on='LOTE', how='outer')

In [ ]:
pysqldf = lambda q: psql(q, globals())

query = '''
    SELECT
        l."LOTE",
        AVG(b.Agitation_PV) AS avg_Agitation_PV,
        MIN(b.Agitation_PV) AS min_Agitation_PV,
        MAX(b.Agitation_PV) AS max_Agitation_PV,

        AVG(b.Air_Sparge_PV) AS avg_Air_Sparge_PV,
        MIN(b.Air_Sparge_PV) AS min_Air_Sparge_PV,
        MAX(b.Air_Sparge_PV) AS max_Air_Sparge_PV,
        
        AVG(b.Biocontainer_Pressure_PV) AS avg_Biocontainer_Pressure_PV,
        MIN(b.Biocontainer_Pressure_PV) AS min_Biocontainer_Pressure_PV,
        MAX(b.Biocontainer_Pressure_PV) AS max_Biocontainer_Pressure_PV,

        AVG(b.DO_1_PV) AS avg_DO_1_PV,
        MIN(b.DO_1_PV) AS min_DO_1_PV,
        MAX(b.DO_1_PV) AS max_DO_1_PV,

        AVG(b.DO_2_PV) AS avg_DO_2_PV,
        MIN(b.DO_2_PV) AS min_DO_2_PV,
        MAX(b.DO_2_PV) AS max_DO_2_PV,

        AVG(b.Gas_Overlay_PV) AS avg_Gas_Overlay_PV,
        MIN(b.Gas_Overlay_PV) AS min_Gas_Overlay_PV,
        MAX(b.Gas_Overlay_PV) AS max_Gas_Overlay_PV,

        AVG(b.Load_Cell_Net_PV) AS avg_Load_Cell_Net_PV,
        MIN(b.Load_Cell_Net_PV) AS min_Load_Cell_Net_PV,
        MAX(b.Load_Cell_Net_PV) AS max_Load_Cell_Net_PV,

        AVG(b.pH_1_PV) AS avg_pH_1_PV,
        MIN(b.pH_1_PV) AS min_pH_1_PV,
        MAX(b.pH_1_PV) AS max_pH_1_PV,

        AVG(b.pH_2_PV) AS avg_pH_2_PV,
        MIN(b.pH_2_PV) AS min_pH_2_PV,
        MAX(b.pH_2_PV) AS max_pH_2_PV,

        AVG(b.PUMP_1_PV) AS avg_PUMP_1_PV,
        MIN(b.PUMP_1_PV) AS min_PUMP_1_PV,
        MAX(b.PUMP_1_PV) AS max_PUMP_1_PV,

        AVG(b.PUMP_1_TOTAL) AS avg_PUMP_1_TOTAL,
        MIN(b.PUMP_1_TOTAL) AS min_PUMP_1_TOTAL,
        MAX(b.PUMP_1_TOTAL) AS max_PUMP_1_TOTAL,

        AVG(b.PUMP_2_PV) AS avg_PUMP_2_PV,
        MIN(b.PUMP_2_PV) AS min_PUMP_2_PV,
        MAX(b.PUMP_2_PV) AS max_PUMP_2_PV,

        AVG(b.PUMP_2_TOTAL) AS avg_PUMP_2_TOTAL,
        MIN(b.PUMP_2_TOTAL) AS min_PUMP_2_TOTAL,
        MAX(b.PUMP_2_TOTAL) AS max_PUMP_2_TOTAL,

        AVG(b.Single_Use_DO_PV) AS avg_Single_Use_DO_PV,
        MIN(b.Single_Use_DO_PV) AS min_Single_Use_DO_PV,
        MAX(b.Single_Use_DO_PV) AS max_Single_Use_DO_PV,

        AVG(b.Single_Use_pH_PV) AS avg_Single_Use_pH_PV,
        MIN(b.Single_Use_pH_PV) AS min_Single_Use_pH_PV,
        MAX(b.Single_Use_pH_PV) AS max_Single_Use_pH_PV,

        AVG(b.Temperatura_PV) AS avg_Temperatura_PV,
        MIN(b.Temperatura_PV) AS min_Temperatura_PV,
        MAX(b.Temperatura_PV) AS max_Temperatura_PV
    FROM
        df_final l
    LEFT JOIN
        df_biopequenyos_02 b
    ON
        l."ID bioreactor" = b.Numero_Biorreactor
        AND b.DateTime BETWEEN l."Fecha/hora inicio" AND l."Fecha/hora fin"
    GROUP BY
        l."LOTE";
'''

resultado = pysqldf(query)

In [ ]:
consulta = """
    SELECT *
    FROM df_final
    LEFT JOIN resultado
    ON df_final.LOTE = resultado.LOTE
    """

In [ ]:
resultadofinal = psql.sqldf(consulta, locals())

In [ ]:
resultadofinal.drop(columns=['LOTE.1'], inplace=True)

In [ ]:
resultadofinal['horas_totales_inoculo'] = (resultadofinal['Fecha/hora fin'] - resultadofinal['Fecha/hora inicio']).dt.total_seconds() / 3600

In [ ]:
resultadofinal = resultadofinal.drop(['Fecha/hora inicio', 'Fecha/hora fin'], axis=1)

In [ ]:
resultadofinal.to_excel(f'{path_cleansed}inoculo_total.xlsx', index=False)

## 3.3 CULTIVO FINAL


In [4]:
# Leer el archivo Excel y guardar el contenido en un DataFrame
path_cultivo_02 = path_cleansed + 'Fases producción_Cultivo final.xlsx'
path_biograndes_02 = path_cleansed + 'biorreactores_grandes.xlsx'
path_cultivoIPC_02 = path_cleansed + 'CineticosIPC_Cultivos.xlsx'

df_cultivo_02 = pd.read_excel(path_cultivo_02)
df_biograndes_02 = pd.read_excel(path_biograndes_02)
df_cultivoIPC_02 = pd.read_excel(path_cultivoIPC_02)

In [5]:
display(df_cultivo_02)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1
0,23019,1,NaN,14615,2023-03-21 07:30:00,2023-03-23 06:30:00,82.40,17.28,91.20,184000000.0,17825,NaN,NaN,1747.920
1,23020,1,NaN,14616,2023-03-21 07:30:00,2023-03-23 06:30:00,80.40,18.80,91.20,181600000.0,14246,NaN,NaN,1676.160
2,23021,1,NaN,13170,2023-03-22 07:30:00,2023-03-24 06:30:00,66.40,16.16,86.40,248000000.0,17825,NaN,NaN,1928.496
3,23022,1,NaN,14614,2023-03-22 07:30:00,2023-03-24 06:30:00,85.60,18.48,83.20,229600000.0,12912,NaN,NaN,1782.800
4,23023,1,NaN,14615,2023-03-28 07:27:00,2023-03-30 10:00:00,77.60,17.12,74.40,132800000.0,17825,26.56,20.88,1861.840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,24049,1,NaN,14617,2024-03-16 09:22:00,2024-03-18 08:23:00,83.60,18.88,72.64,164800000.0,12912,30.56,17.00,1342.800
148,24050,1,NaN,14614,2024-03-23 08:57:00,2024-03-25 08:28:00,84.16,17.76,67.60,152000000.0,6379,29.44,26.64,1422.800
149,24051,1,NaN,13169,2024-03-23 08:57:00,2024-03-25 08:33:00,84.16,17.76,80.80,160800000.0,12912,33.44,19.32,1486.560
150,24052,2,24050.0,14614,2024-03-25 13:28:00,2024-03-27 08:51:00,86.40,17.28,69.04,148000000.0,14246,23.68,18.20,1857.280


In [6]:
query = '''
    SELECT
        l.*,
        AVG(b.Agitation_PV) AS avg_Agitation_PV,
        MIN(b.Agitation_PV) AS min_Agitation_PV,
        MAX(b.Agitation_PV) AS max_Agitation_PV,

        AVG(b.Air_Sparge_PV) AS avg_Air_Sparge_PV,
        MIN(b.Air_Sparge_PV) AS min_Air_Sparge_PV,
        MAX(b.Air_Sparge_PV) AS max_Air_Sparge_PV,

        AVG(b.Biocontainer_Pressure_PV) AS avg_Biocontainer_Pressure_PV,
        MIN(b.Biocontainer_Pressure_PV) AS min_Biocontainer_Pressure_PV,
        MAX(b.Biocontainer_Pressure_PV) AS max_Biocontainer_Pressure_PV,

        AVG(b.DO_1_PV) AS avg_DO_1_PV,
        MIN(b.DO_1_PV) AS min_DO_1_PV,
        MAX(b.DO_1_PV) AS max_DO_1_PV,

        AVG(b.DO_2_PV) AS avg_DO_2_PV,
        MIN(b.DO_2_PV) AS min_DO_2_PV,
        MAX(b.DO_2_PV) AS max_DO_2_PV,

        AVG(b.Gas_Overlay_PV) AS avg_Gas_Overlay_PV,
        MIN(b.Gas_Overlay_PV) AS min_Gas_Overlay_PV,
        MAX(b.Gas_Overlay_PV) AS max_Gas_Overlay_PV,

        AVG(b.Load_Cell_Net_PV) AS avg_Load_Cell_Net_PV,
        MIN(b.Load_Cell_Net_PV) AS min_Load_Cell_Net_PV,
        MAX(b.Load_Cell_Net_PV) AS max_Load_Cell_Net_PV,

        AVG(b.pH_1_PV) AS avg_pH_1_PV,
        MIN(b.pH_1_PV) AS min_pH_1_PV,
        MAX(b.pH_1_PV) AS max_pH_1_PV,

        AVG(b.pH_2_PV) AS avg_pH_2_PV,
        MIN(b.pH_2_PV) AS min_pH_2_PV,
        MAX(b.pH_2_PV) AS max_pH_2_PV,

        AVG(b.PUMP_1_PV) AS avg_PUMP_1_PV,
        MIN(b.PUMP_1_PV) AS min_PUMP_1_PV,
        MAX(b.PUMP_1_PV) AS max_PUMP_1_PV,

        AVG(b.PUMP_1_TOTAL) AS avg_PUMP_1_TOTAL,
        MIN(b.PUMP_1_TOTAL) AS min_PUMP_1_TOTAL,
        MAX(b.PUMP_1_TOTAL) AS max_PUMP_1_TOTAL,

        AVG(b.PUMP_2_PV) AS avg_PUMP_2_PV,
        MIN(b.PUMP_2_PV) AS min_PUMP_2_PV,
        MAX(b.PUMP_2_PV) AS max_PUMP_2_PV,

        AVG(b.PUMP_2_TOTAL) AS avg_PUMP_2_TOTAL,
        MIN(b.PUMP_2_TOTAL) AS min_PUMP_2_TOTAL,
        MAX(b.PUMP_2_TOTAL) AS max_PUMP_2_TOTAL,

        AVG(b.Single_Use_DO_PV) AS avg_Single_Use_DO_PV,
        MIN(b.Single_Use_DO_PV) AS min_Single_Use_DO_PV,
        MAX(b.Single_Use_DO_PV) AS max_Single_Use_DO_PV,

        AVG(b.Single_Use_pH_PV) AS avg_Single_Use_pH_PV,
        MIN(b.Single_Use_pH_PV) AS min_Single_Use_pH_PV,
        MAX(b.Single_Use_pH_PV) AS max_Single_Use_pH_PV,

        AVG(b.Temperatura_PV) AS avg_Temperatura_PV,
        MIN(b.Temperatura_PV) AS min_Temperatura_PV,
        MAX(b.Temperatura_PV) AS max_Temperatura_PV
    FROM
        df_cultivo_02 l
    LEFT JOIN
        df_biograndes_02 b
    ON
        l."ID Bioreactor" = b.Numero_Biorreactor
        AND b.DateTime BETWEEN l."Fecha/hora inicio" AND l."Fecha/hora fin"
    GROUP BY
        l."LOTE";
'''

# Ejecutar la consulta
df_bio_metrics = psql.sqldf(query, locals())

# Ver el resultado
display(df_bio_metrics)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,...,max_PUMP_2_TOTAL,avg_Single_Use_DO_PV,min_Single_Use_DO_PV,max_Single_Use_DO_PV,avg_Single_Use_pH_PV,min_Single_Use_pH_PV,max_Single_Use_pH_PV,avg_Temperatura_PV,min_Temperatura_PV,max_Temperatura_PV
0,23019,1,NaN,14615,2023-03-21 07:30:00.000000,2023-03-23 06:30:00.000000,82.40,17.28,91.20,184000000.0,...,72.465216,1.093214,0.086804,2.883038,18.778709,0.000000,64.813263,80.000000,80.000000,80.000000
1,23020,1,NaN,14616,2023-03-21 07:30:00.000000,2023-03-23 06:30:00.000000,80.40,18.80,91.20,181600000.0,...,7664.883277,21.577901,5.735131,72.934606,5.827639,5.815979,5.864000,29.602002,29.536005,29.664001
2,23021,1,NaN,13170,2023-03-22 07:30:00.000000,2023-03-24 06:30:00.000000,66.40,16.16,86.40,248000000.0,...,7907.745313,683.020051,70.887667,686.276074,791.407652,-0.152000,799.839990,29.602256,29.528003,29.680005
3,23022,1,NaN,14614,2023-03-22 07:30:00.000000,2023-03-24 06:30:00.000000,85.60,18.48,83.20,229600000.0,...,7878.989844,680.947528,65.397822,684.221729,795.767927,6.160000,799.967969,29.576063,28.768005,30.240449
4,23023,1,NaN,14615,2023-03-28 07:27:00.000000,2023-03-30 10:00:00.000000,77.60,17.12,74.40,132800000.0,...,77.339917,0.902589,-0.075232,2.881946,9.226358,0.000000,64.505910,80.000000,80.000000,80.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,24049,1,NaN,14617,2024-03-16 09:22:00.000000,2024-03-18 08:23:00.000000,83.60,18.88,72.64,164800000.0,...,6015.101953,21.457989,7.061660,68.151521,5.624023,5.624023,5.624023,29.600751,29.552002,29.671997
148,24050,1,NaN,14614,2024-03-23 08:57:00.000000,2024-03-25 08:28:00.000000,84.16,17.76,67.60,152000000.0,...,5185.023828,21.856573,7.577548,70.228949,5.771662,5.754651,5.826556,29.601110,29.504004,29.656006
149,24051,1,NaN,13169,2024-03-23 08:57:00.000000,2024-03-25 08:33:00.000000,84.16,17.76,80.80,160800000.0,...,7012.593143,18.256878,7.483303,67.568395,5.840148,5.832000,5.888000,29.600965,29.362171,29.895999
150,24052,2,24050.0,14614,2024-03-25 13:28:00.000000,2024-03-27 08:51:00.000000,86.40,17.28,69.04,148000000.0,...,5945.302734,23.239683,6.838995,69.054651,5.754224,5.616000,5.832000,29.554171,23.797557,30.295996


In [32]:
# Convertimos los campos de fecha de object a datetime
df_bio_metrics['Fecha/hora inicio'] = pd.to_datetime(df_bio_metrics['Fecha/hora inicio'])
df_bio_metrics['Fecha/hora fin'] = pd.to_datetime(df_bio_metrics['Fecha/hora fin'])

# Calculamos la diferencia entre las fechas
df_bio_metrics['horas_totales_cultivofinal'] = (df_bio_metrics['Fecha/hora fin'] - df_bio_metrics['Fecha/hora inicio']).dt.total_seconds() / 3600

In [33]:
#CREAMOS UN DATAFRAME CON MÉTRICAS DE LAS DISTINTAS COLUMNAS DE CINÉTICOS IPC SEGÚN EL LOTE (MEDIA, MIN,...)
df_metricsIPC = df_cultivoIPC_02.groupby('Lote').agg({
    'Turbidez': ['mean', 'std', 'min', 'max'],
    'Viabilidad': ['mean', 'std', 'min', 'max'],
    'Glucosa': ['mean', 'std', 'min', 'max']
}).reset_index()
df_metricsIPC.columns = [''.join(col).strip() for col in df_metricsIPC.columns.values]

In [42]:
query = '''
SELECT 
    b.LOTE,
    `Orden en el encadenado` AS Orden,
    `LOTE parental` AS LOTE_parental,
    `ID Bioreactor` AS ID_Bioreactor,
    horas_totales_cultivofinal,
    `Volumen de inóculo utilizado` AS Volumen_inoculo_utilizado,
    `Turbidez inicio cultivo` AS Turbidez_inicio_cultivo,
    `Turbidez fin cultivo` AS Turbidez_fin_cultivo,
    `Viabilidad final cultivo` AS Viabilidad_final_cultivo,
    `ID Centrífuga` AS ID_Centrifuga,
    `Centrifugación 1 turbidez` AS Centrifugacion_1_turbidez,
    `Centrifugación 2 turbidez` AS Centrifugacion_2_turbidez,
    `Producto 1` AS Producto_1,
    avg_Agitation_PV AS Biorreactor_grande_avg_Agitation_PV,
    min_Agitation_PV AS Biorreactor_grande_min_Agitation_PV,
    max_Agitation_PV AS Biorreactor_grande_max_Agitation_PV,
    avg_Air_Sparge_PV AS Biorreactor_grande_avg_Air_Sparge_PV,
    min_Air_Sparge_PV AS Biorreactor_grande_min_Air_Sparge_PV,
    max_Air_Sparge_PV AS Biorreactor_grande_max_Air_Sparge_PV,
    avg_Biocontainer_Pressure_PV AS Biorreactor_grande_avg_Biocontainer_Pressure_PV,
    min_Biocontainer_Pressure_PV AS Biorreactor_grande_min_Biocontainer_Pressure_PV,
    max_Biocontainer_Pressure_PV AS Biorreactor_grande_max_Biocontainer_Pressure_PV,
    avg_DO_1_PV AS Biorreactor_grande_avg_DO_1_PV,
    min_DO_1_PV AS Biorreactor_grande_min_DO_1_PV,
    max_DO_1_PV AS Biorreactor_grande_max_DO_1_PV,
    avg_DO_2_PV AS Biorreactor_grande_avg_DO_2_PV,
    min_DO_2_PV AS Biorreactor_grande_min_DO_2_PV,
    max_DO_2_PV AS Biorreactor_grande_max_DO_2_PV,
    avg_Gas_Overlay_PV AS Biorreactor_grande_avg_Gas_Overlay_PV,
    min_Gas_Overlay_PV AS Biorreactor_grande_min_Gas_Overlay_PV,
    max_Gas_Overlay_PV AS Biorreactor_grande_max_Gas_Overlay_PV,
    avg_Load_Cell_Net_PV AS Biorreactor_grande_avg_Load_Cell_Net_PV,
    min_Load_Cell_Net_PV AS Biorreactor_grande_min_Load_Cell_Net_PV,
    max_Load_Cell_Net_PV AS Biorreactor_grande_max_Load_Cell_Net_PV,
    avg_pH_1_PV AS Biorreactor_grande_avg_pH_1_PV,
    min_pH_1_PV AS Biorreactor_grande_min_pH_1_PV,
    max_pH_1_PV AS Biorreactor_grande_max_pH_1_PV,
    avg_pH_2_PV AS Biorreactor_grande_avg_pH_2_PV,
    min_pH_2_PV AS Biorreactor_grande_min_pH_2_PV,
    max_pH_2_PV AS Biorreactor_grande_max_pH_2_PV,
    avg_PUMP_1_PV AS Biorreactor_grande_avg_PUMP_1_PV,
    min_PUMP_1_PV AS Biorreactor_grande_min_PUMP_1_PV,
    max_PUMP_1_PV AS Biorreactor_grande_max_PUMP_1_PV,
    avg_PUMP_1_TOTAL AS Biorreactor_grande_avg_PUMP_1_TOTAL,
    min_PUMP_1_TOTAL AS Biorreactor_grande_min_PUMP_1_TOTAL,
    max_PUMP_1_TOTAL AS Biorreactor_grande_max_PUMP_1_TOTAL,
    avg_PUMP_2_PV AS Biorreactor_grande_avg_PUMP_2_PV,
    min_PUMP_2_PV AS Biorreactor_grande_min_PUMP_2_PV,
    max_PUMP_2_PV AS Biorreactor_grande_max_PUMP_2_PV,
    avg_PUMP_2_TOTAL AS Biorreactor_grande_avg_PUMP_2_TOTAL,
    min_PUMP_2_TOTAL AS Biorreactor_grande_min_PUMP_2_TOTAL,
    max_PUMP_2_TOTAL AS Biorreactor_grande_max_PUMP_2_TOTAL,
    avg_Single_Use_DO_PV AS Biorreactor_grande_avg_Single_Use_DO_PV,
    min_Single_Use_DO_PV AS Biorreactor_grande_min_Single_Use_DO_PV,
    max_Single_Use_DO_PV AS Biorreactor_grande_max_Single_Use_DO_PV,
    avg_Single_Use_pH_PV AS Biorreactor_grande_avg_Single_Use_pH_PV,
    min_Single_Use_pH_PV AS Biorreactor_grande_min_Single_Use_pH_PV,
    max_Single_Use_pH_PV AS Biorreactor_grande_max_Single_Use_pH_PV,
    avg_Temperatura_PV AS Biorreactor_grande_avg_Temperatura_PV,
    min_Temperatura_PV AS Biorreactor_grande_min_Temperatura_PV,
    max_Temperatura_PV AS Biorreactor_grande_max_Temperatura_PV,
    Turbidezmean AS IPC_Cultivos_Turbidezmean,
    Turbidezstd AS IPC_Cultivos_Turbidezstd,
    Turbidezmin AS IPC_Cultivos_Turbidezmin,
    Turbidezmax AS IPC_Cultivos_Turbidezmax,
    Viabilidadmean AS IPC_Cultivos_Viabilidadmean,
    Viabilidadstd AS IPC_Cultivos_Viabilidadstd,
    Viabilidadmin AS IPC_Cultivos_Viabilidadmin,
    Viabilidadmax AS IPC_Cultivos_Viabilidadmax,
    Glucosamean AS IPC_Cultivos_Glucosamean,
    Glucosastd AS IPC_Cultivos_Glucosastd,
    Glucosamin AS IPC_Cultivos_Glucosamin,
    Glucosamax AS IPC_Cultivos_Glucosamax

FROM 
    df_bio_metrics b
LEFT JOIN 
    df_metricsIPC m
ON 
    b."LOTE" = m."Lote"
'''

# Ejecutar la consulta
df_cultivo_metrics = psql.sqldf(query, locals())

display(df_cultivo_metrics)

,LOTE,Orden,LOTE_parental,ID_Bioreactor,horas_totales_cultivofinal,Volumen_inoculo_utilizado,Turbidez_inicio_cultivo,Turbidez_fin_cultivo,Viabilidad_final_cultivo,ID_Centrifuga,...,IPC_Cultivos_Turbidezmin,IPC_Cultivos_Turbidezmax,IPC_Cultivos_Viabilidadmean,IPC_Cultivos_Viabilidadstd,IPC_Cultivos_Viabilidadmin,IPC_Cultivos_Viabilidadmax,IPC_Cultivos_Glucosamean,IPC_Cultivos_Glucosastd,IPC_Cultivos_Glucosamin,IPC_Cultivos_Glucosamax
0,23019,1,NaN,14615,47.000000,82.40,17.28,91.20,184000000.0,17825,...,17.28,91.20,9.498000e+07,7.196382e+07,8.333333e+06,1.853333e+08,1.439760,0.548857,0.2056,1.968
1,23020,1,NaN,14616,47.000000,80.40,18.80,91.20,181600000.0,14246,...,18.80,91.20,4.833216e+08,1.250238e+09,7.456000e+06,4.036267e+09,1.471280,0.523726,0.3128,1.944
2,23021,1,NaN,13170,47.000000,66.40,16.16,86.40,248000000.0,17825,...,16.16,86.40,9.388606e+07,8.116626e+07,8.906667e+06,2.450667e+08,1.556582,0.501588,0.2824,2.040
3,23022,1,NaN,14614,47.000000,85.60,18.48,83.20,229600000.0,12912,...,18.48,83.20,1.020812e+08,8.139261e+07,7.853333e+06,2.288000e+08,1.604218,0.494893,0.3664,2.048
4,23023,1,NaN,14615,50.550000,77.60,17.12,74.40,132800000.0,17825,...,17.12,81.60,1.057858e+08,7.315154e+07,5.802667e+06,2.120000e+08,1.416933,0.675252,0.4592,2.192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,24049,1,NaN,14617,47.016667,83.60,18.88,72.64,164800000.0,12912,...,18.88,72.64,1.121049e+08,9.219784e+07,5.648000e+06,1.661333e+08,1.122667,0.504021,0.8080,1.704
148,24050,1,NaN,14614,47.516667,84.16,17.76,67.60,152000000.0,6379,...,17.76,67.60,1.209907e+08,7.757540e+07,5.562667e+06,1.733333e+08,1.274667,0.526077,0.9280,1.880
149,24051,1,NaN,13169,47.600000,84.16,17.76,80.80,160800000.0,12912,...,17.76,80.80,1.273567e+08,8.235297e+07,5.160000e+06,1.850667e+08,1.008000,0.437959,0.7200,1.512
150,24052,2,24050.0,14614,43.383333,86.40,17.28,69.04,148000000.0,14246,...,17.28,71.76,9.459200e+07,5.773566e+07,8.506667e+06,1.477333e+08,1.339200,0.364180,0.9840,1.880


In [53]:
# Consulta SQL para añadir la columna 'lote_abuelo'
query = """
    SELECT df.*, 
           CASE 
               WHEN Orden = 3 THEN (
                   SELECT LOTE_parental
                   FROM df_cultivo_metrics AS parent
                   WHERE parent.lote = df.LOTE_parental
               )
               ELSE NULL
           END AS lote_abuelo
    FROM df_cultivo_metrics df
"""

# Ejecutar la consulta
df_cultivo_03 = psql.sqldf(query, locals())

# Mostrar el resultado
display(df_cultivo_03)

,LOTE,Orden,LOTE_parental,ID_Bioreactor,horas_totales_cultivofinal,Volumen_inoculo_utilizado,Turbidez_inicio_cultivo,Turbidez_fin_cultivo,Viabilidad_final_cultivo,ID_Centrifuga,...,IPC_Cultivos_Turbidezmax,IPC_Cultivos_Viabilidadmean,IPC_Cultivos_Viabilidadstd,IPC_Cultivos_Viabilidadmin,IPC_Cultivos_Viabilidadmax,IPC_Cultivos_Glucosamean,IPC_Cultivos_Glucosastd,IPC_Cultivos_Glucosamin,IPC_Cultivos_Glucosamax,lote_abuelo
0,23019,1,NaN,14615,47.000000,82.40,17.28,91.20,184000000.0,17825,...,91.20,9.498000e+07,7.196382e+07,8.333333e+06,1.853333e+08,1.439760,0.548857,0.2056,1.968,NaN
1,23020,1,NaN,14616,47.000000,80.40,18.80,91.20,181600000.0,14246,...,91.20,4.833216e+08,1.250238e+09,7.456000e+06,4.036267e+09,1.471280,0.523726,0.3128,1.944,NaN
2,23021,1,NaN,13170,47.000000,66.40,16.16,86.40,248000000.0,17825,...,86.40,9.388606e+07,8.116626e+07,8.906667e+06,2.450667e+08,1.556582,0.501588,0.2824,2.040,NaN
3,23022,1,NaN,14614,47.000000,85.60,18.48,83.20,229600000.0,12912,...,83.20,1.020812e+08,8.139261e+07,7.853333e+06,2.288000e+08,1.604218,0.494893,0.3664,2.048,NaN
4,23023,1,NaN,14615,50.550000,77.60,17.12,74.40,132800000.0,17825,...,81.60,1.057858e+08,7.315154e+07,5.802667e+06,2.120000e+08,1.416933,0.675252,0.4592,2.192,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,24049,1,NaN,14617,47.016667,83.60,18.88,72.64,164800000.0,12912,...,72.64,1.121049e+08,9.219784e+07,5.648000e+06,1.661333e+08,1.122667,0.504021,0.8080,1.704,NaN
148,24050,1,NaN,14614,47.516667,84.16,17.76,67.60,152000000.0,6379,...,67.60,1.209907e+08,7.757540e+07,5.562667e+06,1.733333e+08,1.274667,0.526077,0.9280,1.880,NaN
149,24051,1,NaN,13169,47.600000,84.16,17.76,80.80,160800000.0,12912,...,80.80,1.273567e+08,8.235297e+07,5.160000e+06,1.850667e+08,1.008000,0.437959,0.7200,1.512,NaN
150,24052,2,24050.0,14614,43.383333,86.40,17.28,69.04,148000000.0,14246,...,71.76,9.459200e+07,5.773566e+07,8.506667e+06,1.477333e+08,1.339200,0.364180,0.9840,1.880,NaN


In [31]:
df_cultivo_03.to_excel(f'{path_structured}cultivo_final_total.xlsx', index=False)

## 3.4 CULTIVO FINAL TEST

In [38]:
path_cultivotest_02 = path_cleansed + 'Fases producción_test_Cultivo final.xlsx'

# Leemos los archivos Excel y guardamos el contenido en un DataFrame
df_cultivotest_02 = pd.read_excel(path_cultivotest_02)

In [48]:
query = '''
    SELECT
        l.*,
        AVG(b.Agitation_PV) AS avg_Agitation_PV,
        MIN(b.Agitation_PV) AS min_Agitation_PV,
        MAX(b.Agitation_PV) AS max_Agitation_PV,

        AVG(b.Air_Sparge_PV) AS avg_Air_Sparge_PV,
        MIN(b.Air_Sparge_PV) AS min_Air_Sparge_PV,
        MAX(b.Air_Sparge_PV) AS max_Air_Sparge_PV,

        AVG(b.Biocontainer_Pressure_PV) AS avg_Biocontainer_Pressure_PV,
        MIN(b.Biocontainer_Pressure_PV) AS min_Biocontainer_Pressure_PV,
        MAX(b.Biocontainer_Pressure_PV) AS max_Biocontainer_Pressure_PV,

        AVG(b.DO_1_PV) AS avg_DO_1_PV,
        MIN(b.DO_1_PV) AS min_DO_1_PV,
        MAX(b.DO_1_PV) AS max_DO_1_PV,

        AVG(b.DO_2_PV) AS avg_DO_2_PV,
        MIN(b.DO_2_PV) AS min_DO_2_PV,
        MAX(b.DO_2_PV) AS max_DO_2_PV,

        AVG(b.Gas_Overlay_PV) AS avg_Gas_Overlay_PV,
        MIN(b.Gas_Overlay_PV) AS min_Gas_Overlay_PV,
        MAX(b.Gas_Overlay_PV) AS max_Gas_Overlay_PV,

        AVG(b.Load_Cell_Net_PV) AS avg_Load_Cell_Net_PV,
        MIN(b.Load_Cell_Net_PV) AS min_Load_Cell_Net_PV,
        MAX(b.Load_Cell_Net_PV) AS max_Load_Cell_Net_PV,

        AVG(b.pH_1_PV) AS avg_pH_1_PV,
        MIN(b.pH_1_PV) AS min_pH_1_PV,
        MAX(b.pH_1_PV) AS max_pH_1_PV,

        AVG(b.pH_2_PV) AS avg_pH_2_PV,
        MIN(b.pH_2_PV) AS min_pH_2_PV,
        MAX(b.pH_2_PV) AS max_pH_2_PV,

        AVG(b.PUMP_1_PV) AS avg_PUMP_1_PV,
        MIN(b.PUMP_1_PV) AS min_PUMP_1_PV,
        MAX(b.PUMP_1_PV) AS max_PUMP_1_PV,

        AVG(b.PUMP_1_TOTAL) AS avg_PUMP_1_TOTAL,
        MIN(b.PUMP_1_TOTAL) AS min_PUMP_1_TOTAL,
        MAX(b.PUMP_1_TOTAL) AS max_PUMP_1_TOTAL,

        AVG(b.PUMP_2_PV) AS avg_PUMP_2_PV,
        MIN(b.PUMP_2_PV) AS min_PUMP_2_PV,
        MAX(b.PUMP_2_PV) AS max_PUMP_2_PV,

        AVG(b.PUMP_2_TOTAL) AS avg_PUMP_2_TOTAL,
        MIN(b.PUMP_2_TOTAL) AS min_PUMP_2_TOTAL,
        MAX(b.PUMP_2_TOTAL) AS max_PUMP_2_TOTAL,

        AVG(b.Single_Use_DO_PV) AS avg_Single_Use_DO_PV,
        MIN(b.Single_Use_DO_PV) AS min_Single_Use_DO_PV,
        MAX(b.Single_Use_DO_PV) AS max_Single_Use_DO_PV,

        AVG(b.Single_Use_pH_PV) AS avg_Single_Use_pH_PV,
        MIN(b.Single_Use_pH_PV) AS min_Single_Use_pH_PV,
        MAX(b.Single_Use_pH_PV) AS max_Single_Use_pH_PV,

        AVG(b.Temperatura_PV) AS avg_Temperatura_PV,
        MIN(b.Temperatura_PV) AS min_Temperatura_PV,
        MAX(b.Temperatura_PV) AS max_Temperatura_PV
    FROM
        df_cultivotest_02 l
    LEFT JOIN
        df_biograndes_02 b
    ON
        l."ID Bioreactor" = b.Numero_Biorreactor
        AND b.DateTime BETWEEN l."Fecha/hora inicio" AND l."Fecha/hora fin"
    GROUP BY
        l."LOTE";
'''

# Ejecutar la consulta
df_bio_metrics_test = psql.sqldf(query, locals())

# Ver el resultado
display(df_bio_metrics_test)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,...,max_PUMP_2_TOTAL,avg_Single_Use_DO_PV,min_Single_Use_DO_PV,max_Single_Use_DO_PV,avg_Single_Use_pH_PV,min_Single_Use_pH_PV,max_Single_Use_pH_PV,avg_Temperatura_PV,min_Temperatura_PV,max_Temperatura_PV
0,24054,1,NaN,14616,2024-04-16 08:12:00.000000,2024-04-18 07:28:00.000000,81.60,15.44,85.60,184800000,...,7183.130469,20.465047,5.780021,72.630963,5.841373,5.832000,5.864000,29.599317,28.900426,30.149241
1,24055,1,NaN,14614,2024-04-13 08:18:00.000000,2024-04-15 08:30:00.000000,NaN,14.32,73.68,175200000,...,5964.056641,19.574417,8.183004,68.727490,5.830701,5.818623,5.856000,29.607439,29.447998,29.712000
2,24056,1,NaN,14615,2024-04-13 08:18:00.000000,2024-04-15 08:15:00.000000,NaN,14.56,82.40,168000000,...,67.505614,0.735469,0.167822,2.497107,9.724675,0.000000,64.518463,80.000000,80.000000,80.000000
3,24057,1,NaN,13170,2024-04-16 08:12:00.000000,2024-04-18 07:41:00.000000,82.40,17.76,78.96,180800000,...,6052.645582,21.866613,6.678987,72.080063,5.647705,5.632000,5.672000,29.601990,29.256006,30.195487
4,24058,2,24055.0,14614,2024-04-15 12:28:00.000000,2024-04-17 08:14:00.000000,87.20,18.00,82.40,144800000,...,6206.229297,20.617331,5.016961,65.652309,5.801296,5.640000,5.821400,29.582864,23.949269,30.295996
5,24059,2,24056.0,14615,2024-04-15 12:27:00.000000,2024-04-17 08:22:00.000000,87.20,18.08,78.40,166400000,...,66.821680,0.650898,-0.075161,2.456598,9.577750,0.000000,64.351520,79.265519,45.255750,80.000000
6,24060,1,NaN,13169,2024-04-20 09:19:00.000000,2024-04-22 09:12:00.000000,82.40,25.92,76.32,161600000,...,22260.512500,20.890357,6.048154,80.000000,5.816837,5.544000,5.872000,29.600114,29.519998,29.679996
7,24061,1,NaN,14617,2024-04-20 09:24:00.000000,2024-04-22 09:12:00.000000,81.60,28.56,68.96,148800000,...,6418.196875,22.141503,6.924258,80.000000,5.879980,5.879980,5.879980,29.606440,29.559998,29.640002
8,24062,3,24058.0,14614,2024-04-17 12:05:00.000000,2024-04-19 08:05:00.000000,87.20,18.88,80.00,164800000,...,5995.251563,20.897237,5.302253,68.456116,5.780636,5.616000,5.800000,29.481478,22.792115,30.280005
9,24063,1,NaN,13170,2024-04-23 10:35:00.000000,2024-04-25 11:14:00.000000,NaN,15.52,68.72,180800000,...,6813.283594,21.441140,11.659634,69.549927,5.837177,5.819718,5.884202,29.600949,29.559998,29.640002


In [49]:
# Convertimos los campos de fecha de object a datetime
df_bio_metrics_test['Fecha/hora inicio'] = pd.to_datetime(df_bio_metrics_test['Fecha/hora inicio'])
df_bio_metrics_test['Fecha/hora fin'] = pd.to_datetime(df_bio_metrics_test['Fecha/hora fin'])

# Calculamos la diferencia entre las fechas
df_bio_metrics_test['horas_totales_cultivofinal'] = (df_bio_metrics_test['Fecha/hora fin'] - df_bio_metrics_test['Fecha/hora inicio']).dt.total_seconds() / 3600

In [50]:
#CREAMOS UN DATAFRAME CON MÉTRICAS DE LAS DISTINTAS COLUMNAS DE CINÉTICOS IPC SEGÚN EL LOTE (MEDIA, MIN,...)
df_metricsIPC_test = df_cultivoIPC_02.groupby('Lote').agg({
    'Turbidez': ['mean', 'std', 'min', 'max'],
    'Viabilidad': ['mean', 'std', 'min', 'max'],
    'Glucosa': ['mean', 'std', 'min', 'max']
}).reset_index()
df_metricsIPC_test.columns = [''.join(col).strip() for col in df_metricsIPC_test.columns.values]

In [51]:
query = '''
SELECT 
    b.LOTE,
    `Orden en el encadenado` AS Orden,
    `LOTE parental` AS LOTE_parental,
    `ID Bioreactor` AS ID_Bioreactor,
    horas_totales_cultivofinal,
    `Volumen de inóculo utilizado` AS Volumen_inoculo_utilizado,
    `Turbidez inicio cultivo` AS Turbidez_inicio_cultivo,
    `Turbidez fin cultivo` AS Turbidez_fin_cultivo,
    `Viabilidad final cultivo` AS Viabilidad_final_cultivo,
    `ID Centrífuga` AS ID_Centrifuga,
    `Centrifugación 1 turbidez` AS Centrifugacion_1_turbidez,
    `Centrifugación 2 turbidez` AS Centrifugacion_2_turbidez,
    `Producto 1` AS Producto_1,
    avg_Agitation_PV AS Biorreactor_grande_avg_Agitation_PV,
    min_Agitation_PV AS Biorreactor_grande_min_Agitation_PV,
    max_Agitation_PV AS Biorreactor_grande_max_Agitation_PV,
    avg_Air_Sparge_PV AS Biorreactor_grande_avg_Air_Sparge_PV,
    min_Air_Sparge_PV AS Biorreactor_grande_min_Air_Sparge_PV,
    max_Air_Sparge_PV AS Biorreactor_grande_max_Air_Sparge_PV,
    avg_Biocontainer_Pressure_PV AS Biorreactor_grande_avg_Biocontainer_Pressure_PV,
    min_Biocontainer_Pressure_PV AS Biorreactor_grande_min_Biocontainer_Pressure_PV,
    max_Biocontainer_Pressure_PV AS Biorreactor_grande_max_Biocontainer_Pressure_PV,
    avg_DO_1_PV AS Biorreactor_grande_avg_DO_1_PV,
    min_DO_1_PV AS Biorreactor_grande_min_DO_1_PV,
    max_DO_1_PV AS Biorreactor_grande_max_DO_1_PV,
    avg_DO_2_PV AS Biorreactor_grande_avg_DO_2_PV,
    min_DO_2_PV AS Biorreactor_grande_min_DO_2_PV,
    max_DO_2_PV AS Biorreactor_grande_max_DO_2_PV,
    avg_Gas_Overlay_PV AS Biorreactor_grande_avg_Gas_Overlay_PV,
    min_Gas_Overlay_PV AS Biorreactor_grande_min_Gas_Overlay_PV,
    max_Gas_Overlay_PV AS Biorreactor_grande_max_Gas_Overlay_PV,
    avg_Load_Cell_Net_PV AS Biorreactor_grande_avg_Load_Cell_Net_PV,
    min_Load_Cell_Net_PV AS Biorreactor_grande_min_Load_Cell_Net_PV,
    max_Load_Cell_Net_PV AS Biorreactor_grande_max_Load_Cell_Net_PV,
    avg_pH_1_PV AS Biorreactor_grande_avg_pH_1_PV,
    min_pH_1_PV AS Biorreactor_grande_min_pH_1_PV,
    max_pH_1_PV AS Biorreactor_grande_max_pH_1_PV,
    avg_pH_2_PV AS Biorreactor_grande_avg_pH_2_PV,
    min_pH_2_PV AS Biorreactor_grande_min_pH_2_PV,
    max_pH_2_PV AS Biorreactor_grande_max_pH_2_PV,
    avg_PUMP_1_PV AS Biorreactor_grande_avg_PUMP_1_PV,
    min_PUMP_1_PV AS Biorreactor_grande_min_PUMP_1_PV,
    max_PUMP_1_PV AS Biorreactor_grande_max_PUMP_1_PV,
    avg_PUMP_1_TOTAL AS Biorreactor_grande_avg_PUMP_1_TOTAL,
    min_PUMP_1_TOTAL AS Biorreactor_grande_min_PUMP_1_TOTAL,
    max_PUMP_1_TOTAL AS Biorreactor_grande_max_PUMP_1_TOTAL,
    avg_PUMP_2_PV AS Biorreactor_grande_avg_PUMP_2_PV,
    min_PUMP_2_PV AS Biorreactor_grande_min_PUMP_2_PV,
    max_PUMP_2_PV AS Biorreactor_grande_max_PUMP_2_PV,
    avg_PUMP_2_TOTAL AS Biorreactor_grande_avg_PUMP_2_TOTAL,
    min_PUMP_2_TOTAL AS Biorreactor_grande_min_PUMP_2_TOTAL,
    max_PUMP_2_TOTAL AS Biorreactor_grande_max_PUMP_2_TOTAL,
    avg_Single_Use_DO_PV AS Biorreactor_grande_avg_Single_Use_DO_PV,
    min_Single_Use_DO_PV AS Biorreactor_grande_min_Single_Use_DO_PV,
    max_Single_Use_DO_PV AS Biorreactor_grande_max_Single_Use_DO_PV,
    avg_Single_Use_pH_PV AS Biorreactor_grande_avg_Single_Use_pH_PV,
    min_Single_Use_pH_PV AS Biorreactor_grande_min_Single_Use_pH_PV,
    max_Single_Use_pH_PV AS Biorreactor_grande_max_Single_Use_pH_PV,
    avg_Temperatura_PV AS Biorreactor_grande_avg_Temperatura_PV,
    min_Temperatura_PV AS Biorreactor_grande_min_Temperatura_PV,
    max_Temperatura_PV AS Biorreactor_grande_max_Temperatura_PV,
    Turbidezmean AS IPC_Cultivos_Turbidezmean,
    Turbidezstd AS IPC_Cultivos_Turbidezstd,
    Turbidezmin AS IPC_Cultivos_Turbidezmin,
    Turbidezmax AS IPC_Cultivos_Turbidezmax,
    Viabilidadmean AS IPC_Cultivos_Viabilidadmean,
    Viabilidadstd AS IPC_Cultivos_Viabilidadstd,
    Viabilidadmin AS IPC_Cultivos_Viabilidadmin,
    Viabilidadmax AS IPC_Cultivos_Viabilidadmax,
    Glucosamean AS IPC_Cultivos_Glucosamean,
    Glucosastd AS IPC_Cultivos_Glucosastd,
    Glucosamin AS IPC_Cultivos_Glucosamin,
    Glucosamax AS IPC_Cultivos_Glucosamax

FROM 
    df_bio_metrics_test b
LEFT JOIN 
    df_metricsIPC_test m
ON 
    b."LOTE" = m."Lote"
'''

# Ejecutar la consulta
df_cultivo_metrics_test = psql.sqldf(query, locals())

display(df_cultivo_metrics_test)

,LOTE,Orden,LOTE_parental,ID_Bioreactor,horas_totales_cultivofinal,Volumen_inoculo_utilizado,Turbidez_inicio_cultivo,Turbidez_fin_cultivo,Viabilidad_final_cultivo,ID_Centrifuga,...,IPC_Cultivos_Turbidezmin,IPC_Cultivos_Turbidezmax,IPC_Cultivos_Viabilidadmean,IPC_Cultivos_Viabilidadstd,IPC_Cultivos_Viabilidadmin,IPC_Cultivos_Viabilidadmax,IPC_Cultivos_Glucosamean,IPC_Cultivos_Glucosastd,IPC_Cultivos_Glucosamin,IPC_Cultivos_Glucosamax
0,24054,1,NaN,14616,47.266667,81.60,15.44,85.60,184800000,14246,...,15.44,85.60,1.019991e+08,9.012933e+07,5.997333e+06,1.848000e+08,1.216000,0.699440,0.480,1.872
1,24055,1,NaN,14614,48.200000,NaN,14.32,73.68,175200000,12912,...,14.32,73.68,1.192516e+08,9.598512e+07,8.421333e+06,1.754667e+08,1.096000,0.575389,0.744,1.760
2,24056,1,NaN,14615,47.950000,NaN,14.56,82.40,168000000,14246,...,14.56,82.40,1.189058e+08,9.682924e+07,7.384000e+06,1.816000e+08,1.034667,0.733925,0.560,1.880
3,24057,1,NaN,13170,47.483333,82.40,17.76,78.96,180800000,12912,...,17.76,78.96,9.592444e+07,8.712786e+07,6.706667e+06,1.808000e+08,1.272000,0.589997,0.712,1.888
4,24058,2,24055.0,14614,43.766667,87.20,18.00,82.40,144800000,12912,...,18.00,82.40,1.006267e+08,6.660916e+07,1.264000e+07,1.592000e+08,1.046000,0.441446,0.672,1.640
5,24059,2,24056.0,14615,43.916667,87.20,18.08,78.40,166400000,14246,...,18.08,78.40,1.131993e+08,7.652614e+07,1.279733e+07,1.786667e+08,1.102000,0.342438,0.776,1.472
6,24060,1,NaN,13169,47.883333,82.40,25.92,76.32,161600000,14246,...,25.92,76.32,1.316875e+08,7.156721e+07,8.037333e+06,1.794667e+08,0.824000,0.512281,0.336,1.688
7,24061,1,NaN,14617,47.800000,81.60,28.56,68.96,148800000,12912,...,28.56,68.96,1.221872e+08,6.993458e+07,5.602667e+06,1.762667e+08,1.098000,0.416685,0.744,1.696
8,24062,3,24058.0,14614,44.000000,87.20,18.88,80.00,164800000,12912,...,18.88,84.00,8.998222e+07,6.026312e+07,1.258667e+07,1.650667e+08,1.229333,0.499055,0.536,1.760
9,24063,1,NaN,13170,48.650000,NaN,15.52,68.72,180800000,12912,...,15.52,68.72,1.137691e+08,7.266724e+07,3.032000e+06,1.808000e+08,1.544000,0.383583,1.104,1.808


In [54]:
# Consulta SQL para añadir la columna 'lote_abuelo'
query = """
    SELECT df.*, 
           CASE 
               WHEN Orden = 3 THEN (
                   SELECT LOTE_parental
                   FROM df_cultivo_metrics_test AS parent
                   WHERE parent.lote = df.LOTE_parental
               )
               ELSE NULL
           END AS lote_abuelo
    FROM df_cultivo_metrics_test df
"""

# Ejecutar la consulta
df_cultivo_test_03 = psql.sqldf(query, locals())

# Mostrar el resultado
display(df_cultivo_test_03)

,LOTE,Orden,LOTE_parental,ID_Bioreactor,horas_totales_cultivofinal,Volumen_inoculo_utilizado,Turbidez_inicio_cultivo,Turbidez_fin_cultivo,Viabilidad_final_cultivo,ID_Centrifuga,...,IPC_Cultivos_Turbidezmax,IPC_Cultivos_Viabilidadmean,IPC_Cultivos_Viabilidadstd,IPC_Cultivos_Viabilidadmin,IPC_Cultivos_Viabilidadmax,IPC_Cultivos_Glucosamean,IPC_Cultivos_Glucosastd,IPC_Cultivos_Glucosamin,IPC_Cultivos_Glucosamax,lote_abuelo
0,24054,1,NaN,14616,47.266667,81.60,15.44,85.60,184800000,14246,...,85.60,1.019991e+08,9.012933e+07,5.997333e+06,1.848000e+08,1.216000,0.699440,0.480,1.872,NaN
1,24055,1,NaN,14614,48.200000,NaN,14.32,73.68,175200000,12912,...,73.68,1.192516e+08,9.598512e+07,8.421333e+06,1.754667e+08,1.096000,0.575389,0.744,1.760,NaN
2,24056,1,NaN,14615,47.950000,NaN,14.56,82.40,168000000,14246,...,82.40,1.189058e+08,9.682924e+07,7.384000e+06,1.816000e+08,1.034667,0.733925,0.560,1.880,NaN
3,24057,1,NaN,13170,47.483333,82.40,17.76,78.96,180800000,12912,...,78.96,9.592444e+07,8.712786e+07,6.706667e+06,1.808000e+08,1.272000,0.589997,0.712,1.888,NaN
4,24058,2,24055.0,14614,43.766667,87.20,18.00,82.40,144800000,12912,...,82.40,1.006267e+08,6.660916e+07,1.264000e+07,1.592000e+08,1.046000,0.441446,0.672,1.640,NaN
5,24059,2,24056.0,14615,43.916667,87.20,18.08,78.40,166400000,14246,...,78.40,1.131993e+08,7.652614e+07,1.279733e+07,1.786667e+08,1.102000,0.342438,0.776,1.472,NaN
6,24060,1,NaN,13169,47.883333,82.40,25.92,76.32,161600000,14246,...,76.32,1.316875e+08,7.156721e+07,8.037333e+06,1.794667e+08,0.824000,0.512281,0.336,1.688,NaN
7,24061,1,NaN,14617,47.800000,81.60,28.56,68.96,148800000,12912,...,68.96,1.221872e+08,6.993458e+07,5.602667e+06,1.762667e+08,1.098000,0.416685,0.744,1.696,NaN
8,24062,3,24058.0,14614,44.000000,87.20,18.88,80.00,164800000,12912,...,84.00,8.998222e+07,6.026312e+07,1.258667e+07,1.650667e+08,1.229333,0.499055,0.536,1.760,24055.0
9,24063,1,NaN,13170,48.650000,NaN,15.52,68.72,180800000,12912,...,68.72,1.137691e+08,7.266724e+07,3.032000e+06,1.808000e+08,1.544000,0.383583,1.104,1.808,NaN


In [55]:
df_cultivo_test_03.to_excel(f'{path_structured}cultivo_final_test_total.xlsx', index=False)

## 3.5 CENTRIFUGACIÓN

### Notas sobre los resultados las tablas creadas

- No hay datos sobre el Lote 0 en la tabla *CineticosIPC_Centrifugacion.xlsx*



In [5]:
# Leemos los archivos Excel y guardamos el contenido en un DataFrame
path_cent_02 = path_cleansed + 'Centrifugacion.xlsx'
path_OF_02 = path_cleansed + 'OF_123456.xlsx'
path_centrifugadoras_02 = path_cleansed + 'centrifugadoras.xlsx'
path_centIPC_02 = path_cleansed + 'CineticosIPC_Centrifugacion.xlsx'

df_cent_02 = pd.read_excel(path_cent_02)
df_OF_02 = pd.read_excel(path_OF_02)
df_centrifugadoras_02 = pd.read_excel(path_centrifugadoras_02)
df_centIPC_02 = pd.read_excel(path_centIPC_02)

In [6]:
# Pivotamos la tabla Centrifugacion usando 'Orden' como clave para reorganizar las operaciones y sus correspondientes valores de 'DATETIME'
df_pivoted_cent= df_cent_02.pivot_table(index=['Orden', 'EQUIPO'], columns='Operación', values='DATETIME', aggfunc='first').reset_index()
display(df_pivoted_cent)

Operación,Orden,EQUIPO,Centrifugació 1 fi,Centrifugació 1 ini,Centrifugació 2 fi,Centrifugació 2 ini
0,10005144,14246,2023-05-26 15:26:57,2023-05-26 13:45:41,2023-05-26 18:18:12,2023-05-26 16:56:30
1,10005176,14246,2023-07-07 15:06:24,2023-07-07 11:53:24,2023-07-07 17:58:02,2023-07-07 16:15:02
2,10005183,14246,2023-09-08 16:18:56,2023-09-08 12:48:53,2023-09-08 18:47:19,2023-09-08 17:11:03
3,10005223,17825,2023-11-16 11:46:55,2023-11-16 08:38:56,2023-11-16 14:05:13,2023-11-16 12:36:47
4,10005233,14246,2023-11-23 12:01:12,2023-11-23 08:29:16,2023-11-23 14:24:31,2023-11-23 12:48:13
...,...,...,...,...,...,...
242,200209984,14246,2024-07-29 10:32:08,2024-07-29 07:14:41,2024-07-29 13:02:55,2024-07-29 11:21:19
243,200209985,17825,2024-07-31 11:05:43,2024-07-31 07:26:37,2024-07-31 13:36:41,2024-07-31 11:56:32
244,200210971,17825,2024-07-24 10:29:42,2024-07-24 06:47:07,2024-07-24 13:01:55,2024-07-24 11:18:34
245,200211467,17825,2024-08-01 10:46:06,2024-08-01 07:03:58,2024-08-01 13:13:04,2024-08-01 11:33:41


In [7]:
# Gracias al LEFT JOIN sólo nos quedamos con los lotes que aparecen en la tabla 0F_123456 (no hay que tenerlos en cuenta)
consulta = """
    SELECT 
    Lote,
    EQUIPO AS ID_centrifuga,
    `Centrifugació 1 ini` AS Inicio_Centrifugacion_1,
    `Centrifugació 1 fi` AS Fin_Centrifugacion_1,
    `Centrifugació 2 ini` AS Inicio_Centrifugacion_2,
    `Centrifugació 2 fi` AS Fin_Centrifugacion_2


    FROM df_OF_02 OF 
    LEFT JOIN df_pivoted_cent cent
    ON OF.OF = cent.Orden
    """

# Ejecutamos la consulta y generamos un dataframe
df_OF_cent = psql.sqldf(consulta, locals())
display(df_OF_cent)

,Lote,ID_centrifuga,Inicio_Centrifugacion_1,Fin_Centrifugacion_1,Inicio_Centrifugacion_2,Fin_Centrifugacion_2
0,23019,17825.0,2023-03-23 07:53:57,2023-03-23 10:29:43,2023-03-23 12:13:20,2023-03-23 13:03:38
1,23020,14246.0,2023-03-23 08:00:49,2023-03-23 10:26:25,2023-03-23 12:11:04,2023-03-23 13:34:10
2,23021,17825.0,2023-03-24 08:31:15,2023-03-24 11:53:41,2023-03-24 13:13:11,2023-03-24 14:30:45
3,23022,12912.0,2023-03-24 08:11:53,2023-03-24 11:32:53,2023-03-24 13:09:42,2023-03-24 14:29:39
4,23023,17825.0,2023-03-30 09:29:25,2023-03-30 12:26:49,2023-03-30 13:32:39,2023-03-30 14:52:53
...,...,...,...,...,...,...
202,24108,14246.0,2024-07-08 08:09:55,2024-07-08 11:07:41,2024-07-08 11:49:51,2024-07-08 13:32:56
203,24106,12912.0,2024-07-03 07:43:09,2024-07-03 11:00:11,2024-07-03 11:49:07,2024-07-03 13:19:16
204,24107,14246.0,2024-07-03 07:50:38,2024-07-03 11:08:40,2024-07-03 11:59:14,2024-07-03 13:33:24
205,24109,12912.0,2024-07-05 07:43:06,2024-07-05 11:13:40,2024-07-05 12:01:21,2024-07-05 13:43:20


In [11]:
# Calculamos -por lote- la media, el valor mínimo y el máximo de cada medida de la tabla CineticosIPC_Centrifugacion 
# en cada intervalo de tiempo de ambas centrifugaciones
consulta = '''
    SELECT
        OFc.Lote,
        ID_centrifuga,

        -- Métricas centrifugación 1 --

        Inicio_Centrifugacion_1,
        Fin_Centrifugacion_1,

        AVG(cd1.velocidad_separación) AS avg_velocidad_sep_1,
        MIN(cd1.velocidad_separación) AS min_velocidad_sep_1,
        MAX(cd1.velocidad_separación) AS max_velocidad_sep_1,

        AVG(cd1.presion_agua) AS avg_presion_agua_1,
        MIN(cd1.presion_agua) AS min_presion_agua_1,
        MAX(cd1.presion_agua) AS max_presion_agua_1,
        
        AVG(cd1.contrapresion) AS avg_contrapresion_1,
        MIN(cd1.contrapresion) AS min_contrapresion_1,
        MAX(cd1.contrapresion) AS max_contrapresion_1,

        AVG(cd1.caudal) AS avg_caudal_1,
        MIN(cd1.caudal) AS min_caudal_1,
        MAX(cd1.caudal) AS max_caudal_1,

        AVG(cd1.apertura_valvula_agua) AS avg_apertura_valvula_agua_1,
        MIN(cd1.apertura_valvula_agua) AS min_apertura_valvula_agua_1,
        MAX(cd1.apertura_valvula_agua) AS max_apertura_valvula_agua_1,

        AVG(cd1.descargas_totales) AS avg_descargas_totales_1,
        MIN(cd1.descargas_totales) AS min_descargas_totales_1,
        MAX(cd1.descargas_totales) AS max_descargas_totales_1,

        AVG(cd1.`descargas parciales`) AS avg_descargas_parciales_1,
        MIN(cd1.`descargas parciales`) AS min_descargas_parciales_1,
        MAX(cd1.`descargas parciales`) AS max_descargas_parciales_1,

        -- Métricas centrifugación 2 --

        Inicio_Centrifugacion_2,
        Fin_Centrifugacion_2,

        AVG(cd2.velocidad_separación) AS avg_velocidad_sep_2,
        MIN(cd2.velocidad_separación) AS min_velocidad_sep_2,
        MAX(cd2.velocidad_separación) AS max_velocidad_sep_2,

        AVG(cd2.presion_agua) AS avg_presion_agua_2,
        MIN(cd2.presion_agua) AS min_presion_agua_2,
        MAX(cd2.presion_agua) AS max_presion_agua_2,
        
        AVG(cd2.contrapresion) AS avg_contrapresion_2,
        MIN(cd2.contrapresion) AS min_contrapresion_2,
        MAX(cd2.contrapresion) AS max_contrapresion_2,

        AVG(cd2.caudal) AS avg_caudal_2,
        MIN(cd2.caudal) AS min_caudal_2,
        MAX(cd2.caudal) AS max_caudal_2,

        AVG(cd2.apertura_valvula_agua) AS avg_apertura_valvula_agua_2,
        MIN(cd2.apertura_valvula_agua) AS min_apertura_valvula_agua_2,
        MAX(cd2.apertura_valvula_agua) AS max_apertura_valvula_agua_2,

        AVG(cd2.descargas_totales) AS avg_descargas_totales_2,
        MIN(cd2.descargas_totales) AS min_descargas_totales_2,
        MAX(cd2.descargas_totales) AS max_descargas_totales_2,

        AVG(cd2.`descargas parciales`) AS avg_descargas_parciales_2,
        MIN(cd2.`descargas parciales`) AS min_descargas_parciales_2,
        MAX(cd2.`descargas parciales`) AS max_descargas_parciales_2

    FROM
        df_OF_cent OFc

    LEFT JOIN
        df_centrifugadoras_02 cd1
    ON
        OFc.ID_centrifuga = cd1.EQUIPO
        AND cd1.DateTime BETWEEN OFc.Inicio_Centrifugacion_1 AND OFc.Fin_Centrifugacion_1

    LEFT JOIN
        df_centrifugadoras_02 cd2
    ON
        OFc.ID_centrifuga = cd2.EQUIPO
        AND cd2.DateTime BETWEEN OFc.Inicio_Centrifugacion_2 AND OFc.Fin_Centrifugacion_2

    GROUP BY
        OFc.Lote;
'''

# Ejecutamos la consulta y generamos un dataframe
df_metrics_cent = psql.sqldf(consulta, locals())
display(df_metrics_cent)

,Lote,ID_centrifuga,Inicio_Centrifugacion_1,Fin_Centrifugacion_1,avg_velocidad_sep_1,min_velocidad_sep_1,max_velocidad_sep_1,avg_presion_agua_1,min_presion_agua_1,max_presion_agua_1,...,max_caudal_2,avg_apertura_valvula_agua_2,min_apertura_valvula_agua_2,max_apertura_valvula_agua_2,avg_descargas_totales_2,min_descargas_totales_2,max_descargas_totales_2,avg_descargas_parciales_2,min_descargas_parciales_2,max_descargas_parciales_2
0,0,14246.0,2023-07-07 11:53:24,2023-07-07 15:06:24,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,260.666834,1.192823,-0.004933,3.712791,NaN,NaN,NaN,3336.449750,0.000000,6675.081718
1,23019,17825.0,2023-03-23 07:53:57,2023-03-23 10:29:43,6679.593514,6678.278679,6680.694838,NaN,NaN,NaN,...,255.317259,26.471614,26.175997,26.732428,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,23020,14246.0,2023-03-23 08:00:49,2023-03-23 10:26:25,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,258.705689,2.678881,0.016039,3.545359,NaN,NaN,NaN,6650.426390,6518.431966,6677.551716
3,23021,17825.0,2023-03-24 08:31:15,2023-03-24 11:53:41,6680.760697,6678.192329,6683.191550,NaN,NaN,NaN,...,477.474482,26.528782,26.088005,27.419607,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,23022,12912.0,2023-03-24 08:11:53,2023-03-24 11:32:53,0.000000,0.000000,0.000000,0.357143,0.0,1.0,...,563.939978,2.600954,0.555305,3.259559,NaN,NaN,NaN,6678.741666,6677.179237,6681.402561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,24106,12912.0,2024-07-03 07:43:09,2024-07-03 11:00:11,0.000000,0.000000,0.000000,0.071429,0.0,1.0,...,255.335524,1.112902,-0.005150,3.329689,2.015265,1.198738,3.123587,3569.122378,0.000000,6664.209955
203,24107,14246.0,2024-07-03 07:50:38,2024-07-03 11:08:40,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,340.394400,1.679032,0.007609,3.511598,1.201718,0.107519,1.715353,4687.211868,0.000000,6682.231023
204,24108,14246.0,2024-07-08 08:09:55,2024-07-08 11:07:41,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,263.593067,1.334550,0.008328,3.210500,1.605238,1.442146,1.733981,3950.935798,0.000000,6677.068196
205,24109,12912.0,2024-07-05 07:43:06,2024-07-05 11:13:40,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,257.349788,1.026492,-0.005605,3.077578,2.049252,1.408552,3.090590,4080.770998,0.000000,6666.254359


In [14]:
# Convertimos los campos de fecha de object a datetime
df_metrics_cent['Inicio_Centrifugacion_1'] = pd.to_datetime(df_metrics_cent['Inicio_Centrifugacion_1'])
df_metrics_cent['Fin_Centrifugacion_1'] = pd.to_datetime(df_metrics_cent['Fin_Centrifugacion_1'])
df_metrics_cent['Inicio_Centrifugacion_2'] = pd.to_datetime(df_metrics_cent['Inicio_Centrifugacion_2'])
df_metrics_cent['Fin_Centrifugacion_2'] = pd.to_datetime(df_metrics_cent['Fin_Centrifugacion_2'])

# Calculamos la diferencia entre las fechas
df_metrics_cent['tiempo_centrifugacion_1'] = (df_metrics_cent['Fin_Centrifugacion_1'] - df_metrics_cent['Inicio_Centrifugacion_1']).dt.total_seconds() / 3600
df_metrics_cent['tiempo_centrifugacion_2'] = (df_metrics_cent['Fin_Centrifugacion_2'] - df_metrics_cent['Inicio_Centrifugacion_2']).dt.total_seconds() / 3600


In [16]:
# Eliminamos los lotes de CineticosIPC_Centrifugacion que no se encuentrene en OF_123456 (no hay que tenerlos en cuenta)
consulta = '''
    SELECT
        OF.Lote,
        Centrífuga,
        `Centrifugada (1 o 2)`,
        `Volumen centrifugado (L)`,
        Turbidez

    FROM
        df_OF_02 OF

    LEFT JOIN 
        df_centIPC_02 cIPC
    ON 
        OF.Lote = cIPC.Lote

'''

# Ejecutamos la consulta y generamos un dataframe
df_OF_centIPC = psql.sqldf(consulta, locals())
display(df_OF_centIPC)

,Lote,Centrífuga,Centrifugada (1 o 2),Volumen centrifugado (L),Turbidez
0,23019,17825.0,1.0,200.0,26.80
1,23019,17825.0,1.0,400.0,26.32
2,23019,17825.0,1.0,600.0,28.16
3,23019,17825.0,1.0,800.0,28.08
4,23019,17825.0,1.0,1000.0,28.40
...,...,...,...,...,...
2205,24110,14246.0,1.0,200.0,35.04
2206,24110,14246.0,1.0,1000.0,34.24
2207,24110,14246.0,1.0,2000.0,34.40
2208,24110,14246.0,2.0,100.0,28.00


In [17]:
# Pivotamos la tabla CineticosIPC_Centrifugacion (habiendo eliminado los lotes que no aparecen en OF_123456) 
# usando Lote y Centrífuga como claves para reorganizar las operaciones y sus correspondientes valores de Turbidez 
df_pivoted_centIPC= df_OF_centIPC.pivot_table(index=['Lote', 'Centrífuga'], columns=['Centrifugada (1 o 2)', 'Volumen centrifugado (L)'], values='Turbidez', aggfunc='first').reset_index()

# Renombramos los nombres de las columnas
df_pivoted_centIPC.columns = ['Lote', 'Centrífuga', 'Turbidez_1_100', 'Turbidez_1_200','Turbidez_1_400','Turbidez_1_600','Turbidez_1_800','Turbidez_1_1000','Turbidez_1_1200','Turbidez_1_1400','Turbidez_1_1600','Turbidez_1_1800','Turbidez_1_2000','Turbidez_2_100','Turbidez_2_200','Turbidez_2_300','Turbidez_2_400',]
display(df_pivoted_centIPC)

,Lote,Centrífuga,Turbidez_1_100,Turbidez_1_200,Turbidez_1_400,Turbidez_1_600,Turbidez_1_800,Turbidez_1_1000,Turbidez_1_1200,Turbidez_1_1400,Turbidez_1_1600,Turbidez_1_1800,Turbidez_1_2000,Turbidez_2_100,Turbidez_2_200,Turbidez_2_300,Turbidez_2_400
0,23019,17825.0,NaN,26.80,26.32,28.16,28.08,28.40,27.68,28.56,28.48,28.88,28.16,20.80,20.80,21.60,21.52
1,23020,14246.0,NaN,29.84,28.64,30.88,28.56,27.60,27.76,28.32,27.60,28.48,26.88,12.80,13.04,12.88,12.88
2,23021,17825.0,NaN,28.64,30.40,28.72,28.72,29.20,28.64,29.12,29.52,29.12,28.96,18.48,20.48,17.84,20.24
3,23022,12912.0,NaN,26.32,26.40,26.24,26.40,26.24,26.08,26.32,26.96,26.88,26.56,10.08,10.00,11.04,9.44
4,23023,17825.0,NaN,26.56,26.00,26.24,26.80,27.04,26.24,26.96,26.96,26.56,26.88,21.04,20.96,20.80,20.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,24106,12912.0,NaN,32.40,NaN,NaN,NaN,31.44,NaN,NaN,NaN,NaN,32.64,26.80,NaN,NaN,26.00
202,24107,14246.0,NaN,28.56,NaN,NaN,NaN,28.00,NaN,NaN,NaN,NaN,28.32,22.72,NaN,NaN,23.84
203,24108,14246.0,NaN,29.36,NaN,NaN,NaN,30.00,NaN,NaN,NaN,NaN,29.92,15.44,NaN,NaN,15.36
204,24109,12912.0,NaN,31.84,NaN,NaN,NaN,31.36,NaN,NaN,NaN,NaN,31.84,29.36,NaN,NaN,28.00


In [18]:
# Juntamos CineticosIPC_Centrifugacion pivotada con el dataframe df_metrics_cent 
consulta = '''
    SELECT
        mc.Lote,
        ID_centrifuga,

        -- Centrifugacion 1 --
        tiempo_centrifugacion_1,
        avg_velocidad_sep_1,
        min_velocidad_sep_1,
        max_velocidad_sep_1,
        avg_presion_agua_1,
        min_presion_agua_1,
        max_presion_agua_1,
        avg_contrapresion_1,
        min_contrapresion_1,
        max_contrapresion_1,
        avg_caudal_1,
        min_caudal_1,
        max_caudal_1,
        avg_apertura_valvula_agua_1,
        min_apertura_valvula_agua_1,
        max_apertura_valvula_agua_1,
        avg_descargas_totales_1,
        min_descargas_totales_1,
        max_descargas_totales_1,
        avg_descargas_parciales_1,
        min_descargas_parciales_1,
        max_descargas_parciales_1,
        Turbidez_1_100,
        Turbidez_1_200,
        Turbidez_1_400,
        Turbidez_1_600,
        Turbidez_1_800,
        Turbidez_1_1000,
        Turbidez_1_1200,
        Turbidez_1_1400,
        Turbidez_1_1600,
        Turbidez_1_1800,
        Turbidez_1_2000,

        -- Centrifugacion 2 --
        tiempo_centrifugacion_2,
        avg_velocidad_sep_2,
        min_velocidad_sep_2,
        max_velocidad_sep_2,
        avg_presion_agua_2,
        min_presion_agua_2,
        max_presion_agua_2,
        avg_contrapresion_2,
        min_contrapresion_2,
        max_contrapresion_2,
        avg_caudal_2,
        min_caudal_2,
        max_caudal_2,
        avg_apertura_valvula_agua_2,
        min_apertura_valvula_agua_2,
        max_apertura_valvula_agua_2,
        avg_descargas_totales_2,
        min_descargas_totales_2,
        max_descargas_totales_2,
        avg_descargas_parciales_2,
        min_descargas_parciales_2,
        max_descargas_parciales_2,
        Turbidez_2_100,
        Turbidez_2_200,
        Turbidez_2_300,
        Turbidez_2_400


    FROM
        df_metrics_cent mc

    LEFT JOIN 
        df_pivoted_centIPC cIPC
    ON 
        mc.Lote = cIPC.Lote

'''
# Ejecutar la consulta
df_cent_03 = psql.sqldf(consulta, locals())
display(df_cent_03)

,Lote,ID_centrifuga,tiempo_centrifugacion_1,avg_velocidad_sep_1,min_velocidad_sep_1,max_velocidad_sep_1,avg_presion_agua_1,min_presion_agua_1,max_presion_agua_1,avg_contrapresion_1,...,avg_descargas_totales_2,min_descargas_totales_2,max_descargas_totales_2,avg_descargas_parciales_2,min_descargas_parciales_2,max_descargas_parciales_2,Turbidez_2_100,Turbidez_2_200,Turbidez_2_300,Turbidez_2_400
0,0,14246.0,3.216667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,37.693412,...,NaN,NaN,NaN,3336.449750,0.000000,6675.081718,NaN,NaN,NaN,NaN
1,23019,17825.0,2.596111,6679.593514,6678.278679,6680.694838,NaN,NaN,NaN,3.197993,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.80,20.80,21.60,21.52
2,23020,14246.0,2.426667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,24.421936,...,NaN,NaN,NaN,6650.426390,6518.431966,6677.551716,12.80,13.04,12.88,12.88
3,23021,17825.0,3.373889,6680.760697,6678.192329,6683.191550,NaN,NaN,NaN,3.200649,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.48,20.48,17.84,20.24
4,23022,12912.0,3.350000,0.000000,0.000000,0.000000,0.357143,0.0,1.0,32.800480,...,NaN,NaN,NaN,6678.741666,6677.179237,6681.402561,10.08,10.00,11.04,9.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,24106,12912.0,3.283889,0.000000,0.000000,0.000000,0.071429,0.0,1.0,42.780822,...,2.015265,1.198738,3.123587,3569.122378,0.000000,6664.209955,26.80,NaN,NaN,26.00
203,24107,14246.0,3.300556,0.000000,0.000000,0.000000,0.000000,0.0,0.0,39.817761,...,1.201718,0.107519,1.715353,4687.211868,0.000000,6682.231023,22.72,NaN,NaN,23.84
204,24108,14246.0,2.962778,0.000000,0.000000,0.000000,0.000000,0.0,0.0,34.869814,...,1.605238,1.442146,1.733981,3950.935798,0.000000,6677.068196,15.44,NaN,NaN,15.36
205,24109,12912.0,3.509444,0.000000,0.000000,0.000000,0.000000,0.0,0.0,43.068094,...,2.049252,1.408552,3.090590,4080.770998,0.000000,6666.254359,29.36,NaN,NaN,28.00


In [19]:
# Guardamos el resultado en un archivo Excel
df_cent_03.to_excel(f'{path_structured}centrifugacion_total.xlsx', index=False)

## 3.6 MATERIAS PRIMAS

In [ ]:
path_mp_02 = path_cleansed + 'materias_primas.xlsx'

df_mp_02 = pd.read_excel(path_mp_02)

In [ ]:
# Pivotamos la tabla Centrifugacion usando 'Orden' como clave para reorganizar las operaciones y sus correspondientes valores de 'DATETIME'
df_mp_03= df_mp_02.pivot_table(index='Lote', columns='Material', values='Qty', aggfunc='sum').reset_index()
df_mp_03.columns = ['Lote', 'qty_01', 'qty_02', 'qty_03', 'qty_04', 'qty_05', 'qty_06', 'qty_07', 'qty_08', 'qty_09', 'qty_10', 'qty_11', 'qty_12', 'qty_13']
display(df_mp_03)

In [ ]:
df_mp_03.to_excel(f'{path_structured}materias_primas_total.xlsx', index=False)